In [2]:
!pip install pandas
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.8 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
from langchain_community.llms import Replicate
from google.colab import userdata

api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [9]:
df = pd.read_csv("/content/sample_data/personality_synthetic_dataset.csv")

label_col = "personality_type"
feature_cols = [col for col in df.columns if col != label_col]

dataset = df.sample(5, random_state=42)

In [7]:
parameters = {
  "top_k": 5,
  "top_p": 1.0,
  "max_tokens": 4096,
  "min_tokens": 0,
  "random_seed": None,
  "repetition_penalty": 1.0,
  "temperature": 0.7,
  "stopping_criteria": "length (256 tokens)",
  "stopping_sequence": None
}

llm = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs=parameters
)

In [15]:
for i, row in dataset.iterrows():
    traits = "\n".join([f"- {col}: {round(row[col], 2)}" for col in feature_cols])
    label = row[label_col]

    prompt = f"""
    Here are personality trait scores (0–10) of a person:
    {traits}

    Based on this, classify the person's personality as one of:
    - Introvert
    - Extrovert
    - Ambivert
    Also, explain briefly why based on the traits above.
    Output format:
    Personality: [Type] — [Short Reason]
    """

    response = llm.invoke(prompt)

    print(f"Person {i+1}")
    print("Prompt:\n", prompt.strip())
    print("Granite Output:\n", response.strip())
    print(f"Ground Truth Label: {label}")
    print("-----------------------------\n")

Person 10651
Prompt:
 Here are personality trait scores (0–10) of a person:  
    - social_energy: 6.16
- alone_time_preference: 6.58
- talkativeness: 3.63
- deep_reflection: 4.62
- group_comfort: 6.62
- party_liking: 3.85
- listening_skill: 6.18
- empathy: 5.5
- creativity: 8.37
- organization: 6.26
- leadership: 5.5
- risk_taking: 4.15
- public_speaking_comfort: 5.19
- curiosity: 7.15
- routine_preference: 8.79
- excitement_seeking: 4.68
- friendliness: 6.9
- emotional_stability: 6.16
- planning: 5.68
- spontaneity: 2.61
- adventurousness: 3.47
- reading_habit: 7.96
- sports_interest: 1.78
- online_social_usage: 7.05
- travel_desire: 4.77
- gadget_usage: 2.38
- work_style_collaborative: 4.73
- decision_speed: 6.52
- stress_handling: 5.1

    Based on this, classify the person's personality as one of:
    - Introvert
    - Extrovert
    - Ambivert
    Also, explain briefly why based on the traits above.
    Output format:
    Personality: [Type] — [Short Reason]
Granite Output:
 Perso